# Normal Day Analysis

In [21]:
import pandas as pd
import numpy as np
import ipaddress
import dns.resolver
import dns.reversename
import pygeoip
import matplotlib.pyplot as plt 

In [22]:
datafile='data7.parquet'

In [23]:
gi=pygeoip.GeoIP('./GeoIP.dat')
gi2=pygeoip.GeoIP('./GeoIPASNum.dat')
addr='193.136.73.21'
cc=gi.country_code_by_addr(addr)
org=gi2.org_by_addr(addr)
print(cc,org)

PT AS1930 Fundacao para a Ciencia e a Tecnologia, I.P.


In [24]:
data=pd.read_parquet(datafile)

In [25]:
data.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709948,1232265,192.168.107.196,157.240.212.35,tcp,443,13896,113280
709949,1232398,192.168.107.196,157.240.212.35,tcp,443,45276,419472


In [26]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_normal = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET,axis=1)]

private_dst_normal.head()


,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


In [27]:
grp_private_normal = private_dst_normal.groupby(["proto","port"])['dst_ip'].apply(list)

In [28]:

https_servers = set(grp_private_normal[('tcp', 443)])
print()

dns_servers =  set(grp_private_normal[('udp', 53)])
print()

## Servers 
tcp 443 {'192.168.107.223', '192.168.107.238', '192.168.107.222', '192.168.107.226'} https

udp 53 {'192.168.107.227', '192.168.107.234'} dns


In [29]:
grouped = data.groupby(['src_ip','dst_ip']).sum()

# Calculate the ratio
grouped['ratio'] = grouped['up_bytes'] / grouped['down_bytes']
grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()

timestamp  \
src_ip          dst_ip                      
192.168.107.100 104.16.249.130   94725010   
                104.22.48.100    36323031   
                104.244.42.129  246717524   
                104.26.13.207   100077577   
                104.26.14.51     79996025   
...                                   ...   
192.168.107.99  66.22.229.4      11237553   
                66.254.114.41    10868618   
                88.157.217.145   66107954   
                88.157.217.146  120743878   
                97.107.142.234   30141613   

                                                                            proto  \
src_ip          dst_ip                                                              
192.168.107.100 104.16.249.130  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.22.48.100                         tcptcptcptcptcptcptcptcptcp   
                104.244.42.129  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.26.13.207   tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                104.26.14.51     tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcptcp   
...                                                                           ...   
192.168.107.99  66.22.229.4                                             tcptcptcp   
                66.254.114.41                                           tcptcptcp   
                88.157.217.145  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                88.157.217.146  tcptcptcptcptcptcptcptcptcptcptcptcptcptcptcpt...   
                97.107.142.234                           tcptcptcptcptcptcptcptcp   

                                 port  up_bytes  down_bytes     ratio  
src_ip          dst_ip                                                 
192.168.107.100 104.16.249.130   7974    209457     1727986  0.121215  
                104.22.48.100    3987     93381      814578  0.114637  
                104.244.42.129  21707    512007     4246664  0.120567  
                104.26.13.207    8860    270821     6675043  0.040572  
                104.26.14.51     7088    205239     1890983  0.108536  
...                               ...       ...         ...       ...  
192.168.107.99  66.22.229.4      1329     23809      267000  0.089172  
                66.254.114.41    1329     37602      337169  0.111523  
                88.157.217.145   7974    159654     1506733  0.105960  
                88.157.217.146  14619    379690     4234007  0.089676  
                97.107.142.234   3544     71568      677703  0.105604  

[27120 rows x 6 columns]

In [30]:
mean_ratios = grouped.groupby('src_ip')['ratio'].mean().reset_index()

In [31]:
server_traffic = data[data.apply(lambda x: x['dst_ip'] in dns_servers or x['dst_ip'] in https_servers,axis=1)]


In [32]:
server_traffic.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


In [33]:
servers_grouped = server_traffic.groupby(['dst_ip','src_ip']).sum()

# Calculate the ratio
servers_grouped['ratio'] = servers_grouped['up_bytes'] / servers_grouped['down_bytes']
servers_grouped
# Reset the index to make dst_ip a column again
# grouped = grouped.reset_index()
servers_mean_ratios = servers_grouped.groupby('dst_ip')['ratio'].mean().reset_index()

In [34]:
servers_mean_ratios.sort_values(by="ratio",ascending=False)

,dst_ip,ratio
3,192.168.107.227,0.435827
4,192.168.107.234,0.434916
0,192.168.107.222,0.110127
1,192.168.107.223,0.109864
2,192.168.107.226,0.108967
5,192.168.107.238,0.108939


In [35]:
servers_mean_ratios.to_csv("dst_ip_normal.csv")

In [36]:
NET=ipaddress.IPv4Network('192.168.107.0/24')
private_dst_internal_normal = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) in NET  and  ipaddress.IPv4Address(x['src_ip']) in NET  ,axis=1)]

private_dst_internal_normal.head()

,timestamp,src_ip,dst_ip,proto,port,up_bytes,down_bytes
index,,,,,,,
709945,1232147,192.168.107.196,192.168.107.234,udp,53,157,224
709946,1232151,192.168.107.196,192.168.107.234,udp,53,157,227
709947,1232156,192.168.107.196,192.168.107.234,udp,53,154,219
709954,1232769,192.168.107.196,192.168.107.226,tcp,443,13661,134605
709967,1234167,192.168.107.196,192.168.107.226,tcp,443,5129,39416


In [37]:
private_dst_internal_normal.groupby("src_ip")['dst_ip'].agg(set).reset_index().to_csv("normal_coms_internal.csv")

print(private_dst_internal_normal.groupby("src_ip")['dst_ip'].agg(list).reset_index()[:]['src_ip'][0])
print(set(private_dst_internal_normal.groupby("src_ip")['dst_ip'].agg(list).reset_index()[:]['dst_ip'][0]))

192.168.107.100
{'192.168.107.238', '192.168.107.234', '192.168.107.226', '192.168.107.227', '192.168.107.223', '192.168.107.222'}


In [38]:
exterior = data[data.apply(lambda x: ipaddress.IPv4Address(x['dst_ip']) not in NET,axis=1)]
exterior.loc[:,'ratio'] = exterior['up_bytes']/exterior['down_bytes']

/tmp/ipykernel_3916/4032262708.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  exterior.loc[:,'ratio'] = exterior['up_bytes']/exterior['down_bytes']


In [39]:
grp_exterior_normal = exterior.groupby(["proto","port"])['dst_ip'].apply(list)

https_servers = set(grp_exterior_normal[('tcp', 443)])
print(https_servers)

ips_cc = {addr : gi.country_code_by_addr(addr) for addr in https_servers}
ips_cc_filtered = {k:v for k,v in ips_cc.items() if k not in ['US','GB','UK']}

# dns_servers =  set(grp_exterior_normal[('udp', 53)])
# print(dns_servers)

{'157.55.11.230', '52.108.125.93', '40.77.246.253', '69.164.195.213', '143.42.183.127', '151.148.8.24', '173.255.237.238', '150.222.227.80', '52.245.148.43', '173.255.220.144', '205.251.242.35', '15.230.143.196', '143.42.230.142', '70.37.162.217', '138.197.255.158', '40.77.242.82', '157.55.115.18', '40.77.172.152', '20.195.128.250', '129.149.2.103', '212.71.233.247', '150.222.71.101', '172.104.196.152', '205.251.254.171', '52.108.119.24', '52.126.199.166', '132.145.2.130', '65.55.110.40', '52.114.181.128', '40.123.129.234', '52.123.192.58', '40.82.5.15', '23.103.204.84', '13.34.38.82', '13.87.121.65', '140.204.76.130', '150.222.196.161', '65.55.120.201', '13.107.4.176', '150.222.140.44', '52.239.167.254', '129.149.28.95', '16.12.16.227', '198.211.100.66', '40.96.55.68', '15.230.134.198', '15.230.140.161', '157.55.154.254', '52.113.202.97', '15.230.205.229', '40.123.146.223', '15.248.11.155', '65.55.210.239', '13.107.255.211', '52.219.234.225', '192.48.237.214', '15.230.173.90', '207.13

In [40]:
exterior.groupby(['src_ip','dst_ip'])["ratio"].mean().reset_index().sort_values(ascending=False, by='ratio').to_csv("dst_ip_ratio_normal.csv")